In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
import cv2
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
from sklearn.model_selection import train_test_split
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm, tqdm_notebook
import albumentations as A
from albumentations.pytorch import ToTensorV2
#from torchsummary import summary

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.get_device_name(device)

In [ ]:
pip install pretrainedmodels

In [ ]:
import pretrainedmodels

In [ ]:
pip install pytorch-metric-learning

In [ ]:
from pytorch_metric_learning import losses

# Dataset preparation

In [ ]:
df = pd.read_csv('../input/aptos-csv/train.csv')

In [ ]:
#df['diagnosis'] = df['diagnosis'].apply(lambda x: 0 if x==1 else x)
df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x==1 or x==2 or x==3 or x ==4 else x)

In [ ]:
df['diagnosis'].value_counts()

In [ ]:
image_dir = '../input/cropped-clahe-dr/Cropped_CLAHE_train_images/'
train_df, val_df = train_test_split(df, test_size = 0.2, shuffle=True, random_state= 44, stratify = df['diagnosis'])

In [ ]:
image_dir = '../input/cropped-clahe-dr/Cropped_CLAHE_train_images/'
train_df, val_df = train_test_split(df, test_size = 0.2, shuffle=True, random_state= 44, stratify = df['diagnosis'])

In [ ]:
transform = transforms.Compose([
            transforms.Resize(size=(224, 224), interpolation=Image.NEAREST),
            transforms.ToTensor()
])

In [ ]:
aug = A.Compose([
    #A.RandomBrightnessContrast(brightness_limit=1, contrast_limit=1, p=1.0),
    A.OneOf([
        A.Rotate(limit=90, p=1, border_mode=cv2.BORDER_CONSTANT),
        A.Rotate(limit=270, p=1, border_mode=cv2.BORDER_CONSTANT),
        A.HorizontalFlip(p=1),
        A.VerticalFlip(p=1)
    ], p=1)
])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, image_dir, transform= None, aug = None):
        super(CustomDataset, self).__init__()
        self.image_ids = list(df['id_code'])
        self.labels = list(df['diagnosis'])
        self.image_dir = image_dir
        self.transform = transform
        self.aug = aug

    def __getitem__(self, idx):
        file_name = self.image_ids[idx]
        label = self.labels[idx]
        image = Image.open(self.image_dir+file_name+'.png').convert('RGB')
        aug_image = np.array(image)
        if self.aug:
            aug_image = self.aug(image=aug_image)
            image2 = transforms.ToPILImage()(aug_image['image'])
        if self.transform:
            image2 = self.transform(image2)
            image = self.transform(image)
        label = torch.tensor(label)
        return image, image2, label

    def __len__(self):
        return len(self.image_ids)

In [ ]:
train_dataset = CustomDataset(train_df, image_dir, transform=transform, aug= aug)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=8, pin_memory=True, drop_last=True)

In [ ]:
test_dataset = CustomDataset(val_df, image_dir, transform=transform, aug = aug)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=8, pin_memory=True)

# Xception

In [ ]:
model_name = 'xception'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

In [ ]:
model.last_linear = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU()
)

In [ ]:
class Siamese_Encoder(nn.Module):
    def __init__(self, encoder):
        super(Siamese_Encoder, self).__init__()
        self.encoder = encoder
    
    def forward(self, x):
        return self.encoder(x)
    """
    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2
    """

In [ ]:
Model = Siamese_Encoder(model).to(device) #Xception encoder

# Load Pretrained weights

In [ ]:
trained_100_epoch = torch.load('../input/xception-128-100/Xception_embedding_128_100.pth')

In [ ]:
Model.load_state_dict(trained_100_epoch.state_dict())

In [ ]:
optimizer = torch.optim.Adam(Model.parameters(), lr=0.001)
#criterion = SupervisedContrastiveLoss(temperature=0.07)
criterion = losses.SupConLoss(temperature=0.1)

In [ ]:
training_loss = []
valid_loss_min = np.inf

In [ ]:
import os
os.mkdir("./saved_model/")

In [ ]:
for n in range(100):
    train_loss = 0.0
    Model.train()
    for batch in tqdm_notebook(train_dataloader, leave=False):
        x1, x2, y = batch
        x1 = x1.to(device)
        x2 = x2.to(device)
        y = torch.from_numpy(np.asarray(y)).to(device)
        #print("x1...................", torch.min(x1), torch.max(x1))
        #print("x2...................", torch.min(x2), torch.max(x2))
        out1 = Model(x1)
        out2 = Model(x2)
        #print(out1.shape)
        #print("out1...................", torch.min(out1), torch.max(out1))
        #print("out2...................", torch.min(out2), torch.max(out2))
        features = torch.cat((out1, out2), dim=0).squeeze()
        y = torch.cat((y, y), dim=0)
        #print(features.shape)
        #print(y.shape)
        loss = criterion(features, y)
        #print(loss.item())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    train_loss /= len(train_dataloader.dataset)
    training_loss.append([n,train_loss])
    print("epoch number: ", n)
    print(f"Train Loss: {(100*train_loss):.2f}%")
    
    if valid_loss_min>train_loss:
        torch.save(Model, './saved_model/Xception_embedding_128.pth')
        print("new added", train_loss)
        valid_loss_min = train_loss

In [ ]:
loss_df = pd.DataFrame(np.array(training_loss))

In [ ]:
loss_df.to_csv('./Suploss.csv')

# classification model

In [ ]:
class FinalModel(nn.Module):
    def __init__(self, encoder):
        super(FinalModel, self).__init__()
        self.encoder = encoder
        self.classifier = nn.Sequential(
            nn.Linear(2048, 2)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)
        return x

In [ ]:
encoder2 = torch.load('../input/xception-128-100/Xception_embedding_128_100.pth')
encoder2.encoder.last_linear = nn.Identity()
model = FinalModel(encoder2).to(device)
for param in model.encoder.parameters():
    param.requires_grad = False
model

In [ ]:
learning_rate = 0.001
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [ ]:
Test_acc = 0.0

In [ ]:
training_acc = []
Test_Result = []
for n in range(30):
    total = 0
    train_loss = 0.0
    model.train()
    for batch in tqdm_notebook(train_dataloader, leave=False):
        x1, x2, y = batch
        x1 = x1.to(device)
        del x2
        y = torch.from_numpy(np.asarray(y)).to(device)
        pred = model(x1)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += (pred.argmax(1) == y).type(torch.float).sum().item()
    total /= len(train_dataloader.dataset)
    train_loss /= len(train_dataloader.dataset)
    training_acc.append({'Accuracy': total, 'Avg loss': train_loss})
    
    """evaluation"""
    model.eval()
    size = len(test_dataloader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch in tqdm_notebook(test_dataloader, leave=False):
            x1, x2, y = batch
            x1 = x1.to(device)
            del x2
            y = torch.from_numpy(np.asarray(y)).to(device)
            pred = model(x1)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print("epoch number: ", n)
    print(f"Train Accuracy: {(100*total):.2f}%")
    print(f"Test Result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    Test_Result.append({'Accuracy': correct, 'Avg loss': test_loss})
    if Test_acc<correct:
        torch.save(model, 'Xception_classifier.pth')
        print("new added", train_loss)
        Test_acc = correct

In [ ]:
Test = pd.DataFrame(Test_Result)
Test

In [ ]:
Test.to_csv('./test_result.csv')

In [ ]:
pd.DataFrame(training_acc).to_csv('./training_acc.csv')

In [ ]:
model_100 = torch.load('Xception_classifier.pth')

In [ ]:
model.load_state_dict(model_100.state_dict())

In [ ]:
y_true = []
y_pred = []

In [ ]:
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
model.eval()
size = len(test_dataloader.dataset)
test_loss, correct = 0, 0
with torch.no_grad():
    for batch in tqdm_notebook(test_dataloader, leave=False):
        x1, x2, y = batch
        x1 = x1.to(device)
        del x2
        y = torch.from_numpy(np.asarray(y)).to(device)
        pred = model(x1)
        _, preds = torch.max(pred, 1)
        
        for t, p in zip(y.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        y_pred.append(pred.cpu().numpy())
        y_true.append(y.cpu().numpy())
test_loss /= size
correct /= size
print(f"Test Result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
print(confusion_matrix)

# ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
y = np.concatenate(y_true, axis=0)

In [ ]:
y.shape

In [ ]:
y.dtype

In [ ]:
y_p = np.concatenate(y_pred, axis=0 )

In [ ]:
y_p.shape

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [ ]:
y_pd = []

In [ ]:
for i in range(len(y_p)):
    a = softmax(y_p[i])
    y_pd.append(list(a))

In [ ]:
y_pd = np.array(y_pd)

In [ ]:
y.shape

In [ ]:
y_pd.shape

In [ ]:
y_y_y = y_pd.argmax(1)

In [ ]:
y_y_y.shape

In [ ]:
roc = roc_auc_score(y, y_y_y, multi_class='ovr')
roc

In [ ]:
fpr, tpr, _ = roc_curve(y, y_y_y)

In [ ]:
fpr

In [ ]:
tpr

In [ ]:
t = np.save("tpr.npy", tpr)

In [ ]:
f = np.save("fpr.npy", fpr)

In [ ]:
f = np.load("fpr.npy")

In [ ]:
t = np.load("../input/roc-curve-data/tpr.npy")

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot([0, 1], [0, 1], 'k--')
#ax.set_xlim([0.0, 1.0])
#ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic curve')
#plt.plot([0, 1], [0, 1], ‘^’)
ax.plot(fpr, tpr, linestyle='--', label='( AUC area (SCL method)= {:.3f})'.format(roc))
ax.legend(loc="best")
ax.grid(alpha=.4)
#sns.despine()
plt.savefig('ROC',dpi=500)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic curve')
#plt.plot([0, 1], [0, 1], ‘^’)
ax.plot(fpr, tpr, linestyle='--', label='model with margin_0.1(area= {:.3f})'.format(0.98501))
ax.legend(loc="best")
ax.grid(alpha=.4)
#sns.despine()
plt.savefig('ROC',dpi=500)
plt.show()

In [ ]:
del model

# End-to_End Method

In [ ]:
model_name = 'xception'
model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet')

In [ ]:
model.last_linear = nn.Linear(in_features=2048, out_features=2)

In [ ]:
model.to(device)

In [ ]:
transform = transforms.Compose([
            transforms.Resize(size=(224, 224), interpolation=Image.NEAREST),
            transforms.ToTensor()
])

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, image_dir, transform= None):
        super(CustomDataset, self).__init__()
        self.image_ids = list(df['id_code'])
        self.labels = list(df['diagnosis'])
        self.image_dir = image_dir
        self.transform = transform
        

    def __getitem__(self, idx):
        file_name = self.image_ids[idx]
        label = self.labels[idx]
        image = Image.open(self.image_dir+file_name+'.png').convert('RGB')
        
        if self.transform:
            #image2 = self.transform(image2)
            image = self.transform(image)
        label = torch.tensor(label)
        return image, label

    def __len__(self):
        return len(self.image_ids)

In [ ]:
train_dataset = CustomDataset(train_df, image_dir, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, drop_last=False)

In [ ]:
test_dataset = CustomDataset(val_df, image_dir, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [ ]:
weights = torch.FloatTensor([1/1805, 1/999, 1/370, 1/295, 1/193]).to(device)
learning_rate = 0.001
epochs = 10
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), learning_rate)

In [ ]:
Test = 0.0

In [ ]:
training_acc = []
Test_Result = []
for n in range(30):
    total = 0
    train_loss = 0.0
    model.train()
    for batch in tqdm_notebook(train_dataloader, leave=False):
        x, y = batch
        x = x.to(device)
        y = torch.from_numpy(np.asarray(y)).to(device)
        pred = model(x)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += (pred.argmax(1) == y).type(torch.float).sum().item()
    total /= len(train_dataloader.dataset)
    train_loss /= len(train_dataloader.dataset)
    training_acc.append({'Accuracy': total, 'Avg loss': train_loss})
    
    """evaluation"""
    model.eval()
    size = len(test_dataloader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch in tqdm_notebook(test_dataloader, leave=False):
            x, y = batch
            x = x.to(device)
            y = torch.from_numpy(np.asarray(y)).to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print("epoch number: ", n)
    print(f"Train Accuracy: {(100*total):.2f}%")
    print(f"Test Result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    Test_Result.append({'Accuracy': correct, 'Avg loss': test_loss})
    if Test<correct:
        torch.save(model, "best_classifier_no_scl.pth")
        Test = correct
        print("new added")

In [ ]:
Test = pd.DataFrame(Test_Result)
Test

In [ ]:
Test.to_csv('./test_result_no_scl.csv')

In [ ]:
pd.DataFrame(training_acc).to_csv('./training_acc_no_scl.csv')

In [ ]:
model_100 = torch.load('best_classifier_no_scl.pth')
model.load_state_dict(model_100.state_dict())

In [ ]:
y_true = []
y_pred = []

In [ ]:
nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)
model.eval()
size = len(test_dataloader.dataset)
test_loss, correct = 0, 0
with torch.no_grad():
    for batch in tqdm_notebook(test_dataloader, leave=False):
        x, y = batch
        x = x.to(device)
        y = torch.from_numpy(np.asarray(y)).to(device)
        pred = model(x)
        _, preds = torch.max(pred, 1)
        for t, p in zip(y.view(-1), preds.view(-1)):
            confusion_matrix[t.long(), p.long()] += 1
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
        y_pred.append(pred.cpu().numpy())
        y_true.append(y.cpu().numpy())
test_loss /= size
correct /= size
print(f"Test Result: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
print(confusion_matrix)

In [ ]:
y = np.concatenate(y_true, axis=0)
y_p = np.concatenate(y_pred, axis=0 )

In [ ]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [ ]:
y_pd = []

for i in range(len(y_p)):
    a = softmax(y_p[i])
    y_pd.append(list(a))

In [ ]:
y_pd = np.array(y_pd)
y_y_y = y_pd.argmax(1)

In [ ]:
roc2 = roc_auc_score(y, y_y_y, multi_class='ovr')
roc2

In [ ]:
fpr2, tpr2, _ = roc_curve(y, y_y_y)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
ax.plot([0, 1], [0, 1], 'k--')
#ax.set_xlim([0.0, 1.0])
#ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic curve')
#plt.plot([0, 1], [0, 1], ‘^’)
ax.plot(fpr, tpr, linestyle='--', label='( AUC area (SCL)= {:.3f})'.format(roc))
ax.plot(fpr2, tpr2, linestyle='--', label='( AUC area (W/O SCL)= {:.3f})'.format(roc2))
ax.legend(loc="best")
ax.grid(alpha=.4)
#sns.despine()
plt.savefig('comp.png',dpi=500)
plt.show()

# Visualize with TSNE

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
encoder2 = torch.load('../input/xception-128-100/Xception_embedding_128_100.pth')

In [ ]:
features = []
labels = []
encoder2.eval()
with torch.no_grad():
    for batch in tqdm_notebook(train_dataloader, leave=False):
        x1, x2, y = batch
        x1 = x1.to(device)
        del x2
        y = torch.from_numpy(np.asarray(y)).to(device)
        pred = encoder2(x1)
        labels += list(y.cpu())
        features.append(pred.cpu().numpy())

In [ ]:
len(train_dataloader.dataset)

In [ ]:
X = np.array(features).reshape(-1, 128)

In [ ]:
X.shape

In [ ]:
embedded_space = TSNE(n_components=2).fit_transform(X)

In [ ]:
y = [labels[i].item() for i in range(len(labels))]

In [ ]:
len(y)

In [ ]:
df = pd.DataFrame()
df["y"] = y
df["Dimension 1"] = embedded_space[:,0]
df["Dimension 2"] = embedded_space[:,1]
#df["Dimension 3"] = embedded_space[:, 2]

In [ ]:
s = sns.scatterplot(x="Dimension 1", y="Dimension 2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df)#.set(title="Diabetic Retinopathy Test Data T-SNE projection")

In [ ]:
s.get_figure().savefig("train_points.png")

In [ ]:
vis_x = embedded_space[:, 0]
vis_y = embedded_space[:, 1]
#vis_z = embedded_space[:, 2]

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(vis_x, vis_y, vis_z, c=y, cmap=plt.cm.get_cmap("jet", 10))
ax.view_init(40)
#plt.colorbar(ticks=range(10))
ax.set_xlabel("Dimension 1")
ax.set_ylabel("Dimention 2")
ax.set_zlabel("Dimension 3")
plt.show()

In [ ]:
fig.savefig("./scatterplot.png", dpi=500)

# Seaborn plot

In [ ]:
embedded_space = TSNE(n_components=2).fit_transform(X)
embedded_space.shape

In [ ]:
df = pd.DataFrame()
df["y"] = y
df["Dimension 1"] = embedded_space[:,0]
df["Dimension 2"] = embedded_space[:,1]
#df["Dimension 3"] = embedded_space[:, 2]

In [ ]:
plt.figure()
s = sns.scatterplot(x="Dimension 1", y="Dimension 2", hue=df.y.tolist(),
                palette=sns.color_palette("hls", 2),
                data=df)#.set(title="Diabetic Retinopathy Test Data T-SNE projection")

In [ ]:
s.get_figure().savefig("test_.png")